In [50]:
from math import *
import random
import sys
import numpy as np

In [51]:
current_phases0 = [31.00, 3.00, 17.00, 3.00, 27.00, 3.00, 38.00, 3.00, 52.00, 3.00, 33.00, 3.00, 17.00, 3.00, 22.00, 3.00, 32.00, 3.00, 61.00, 3.00,
                   25.00, 3.00, 47.00, 3.00, 18.00, 3.00, 51.00, 3.00, 24.00, 3.00]

In [52]:
class Particle:
    def __init__(self, max_iter):
        self.position = current_phases0.copy()  # particle current position
        self.velocity = [random.uniform(-1, 1) for _ in range(30)]  # particle current velocity
        self.best_position = current_phases0.copy()  # particle best position
        self.fitness = sys.maxsize   # particle fitness
        self.best_fitness = sys.maxsize  # particle best fitness
        self.iteration = 0 # 반복 횟수
        self.max_iter = max_iter

    def evaluate_fitness(self, fitness_func):
        self.fitness = fitness_func(self.position)
        if self.fitness < self.best_fitness:
            self.best_position = self.position.copy()
            self.best_fitness = self.fitness

    def update_velocity(self, global_best_position):
        w_min = 0.7
        w_max = 1.2
        self.iteration += 1
        w = w_max - ((w_max - w_min) * self.iteration / self.max_iter)
        c1 = 1.5
        c2 = 2.5
        for i in range(len(self.position)):
            r1 = random.random()
            r2 = random.random()
            cognitive_velocity = c1 * r1 * (self.best_position[i] - self.position[i])
            social_velocity = c2 * r2 * (global_best_position[i] - self.position[i])
            self.velocity[i] = w * self.velocity[i] + cognitive_velocity + social_velocity

    def update_position(self, bounds):
        for i in range(len(self.position)):
            self.position[i] += self.velocity[i]
            if self.position[i] <= bounds[i][0]:
                self.position[i] = bounds[i][0]
            elif self.position[i] >= bounds[i][1]:
                self.position[i] = bounds[i][1]

class PSO:
    def __init__(self, fitness_function, bounds, num_particles, max_iter):
        self.fitness_func = fitness_function
        self.bounds = bounds
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.global_best_position = current_phases0.copy()
        self.global_best_fitness = sys.maxsize
        self.swarm = [Particle(max_iter) for _ in range(num_particles)]

    def run_result(self):
        for i in range(self.max_iter):
            for j in range(self.num_particles):
                self.swarm[j].evaluate_fitness(self.fitness_func)
                if self.swarm[j].fitness < self.global_best_fitness:
                    self.global_best_position = self.swarm[j].position.copy()
                    self.global_best_fitness = self.swarm[j].fitness
            for j in range(self.num_particles):
                self.swarm[j].update_velocity(self.global_best_position)
                self.swarm[j].update_position(self.bounds)
        print('Best position:', self.global_best_position)
        print('Best fitness:', self.global_best_fitness)
        return self.global_best_position, self.global_best_fitness


In [70]:
import pandas as pd

In [116]:
result = []
def objective_function(x):

    # if sum(x1) != 120:
    #     return sys.maxsize
    # elif sum(x2) != 120:
    #     return sys.maxsize
    # elif sum(x3) != 120:
    #     return sys.maxsize
    
    # penalty = 1000  # penalty value
    # sum_cond1 = abs(180 - sum(x[:10])) * penalty
    # sum_cond2 = abs(180 - sum(x[10:20])) * penalty
    # sum_cond3 = abs(180 - sum(x[20:30])) * penalty
    
    z = sum(x)  #+ sum_cond1 + sum_cond2 + sum_cond3
    
    result.append([x, z])
    
    if len(result) % 1000 ==0:
        pd.DataFrame(result).to_csv('result{}.csv'.format(len(result)), header=False, index=False)
    
    return z

min_dur = [30, 3, 40, 3, 50, 3, 10, 3, 10, 3, 10, 3, 20, 3, 10, 3, 50, 3, 10, 3,10, 3, 10, 3, 10, 3, 10, 3, 10, 3]
max_dur = [50, 3, 70, 3, 100, 3, 70, 3, 30, 3, 30, 3, 30, 3, 40, 3, 200, 3, 30, 3,30, 3, 30, 3, 30, 3, 50, 3, 40, 3]



In [117]:
bounds = []
for i in range(len(min_dur)):
    bounds.append((min_dur[i], max_dur[i]))


In [118]:
if __name__ == "__main__":
    num_particles = 10
    maxiter = 2000
    pso = PSO(objective_function, bounds, num_particles, maxiter)
    pso.run_result()

Best position: [30, 3, 40, 3, 50, 3, 10, 3, 30, 3, 10, 3, 20, 3, 10, 3, 50, 3, 10, 3, 10, 3, 30, 3, 10, 3, 50, 3, 10, 3]
Best fitness: 415
